In [ ]:
# 1. NoSQL Databases:
#   a. Write a Python program that connects to a MongoDB database and inserts a new document into a collection named "students". The document should include fields such as "name", "age", and "grade". Print a success message after the insertion.


from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
database = client['mydatabase']
collection = database['students']

# Create a new document
document = {
    'name': 'John Doe',
    'age': 18,
    'grade': 'A'
}

# Insert the document into the collection
result = collection.insert_one(document)

# Print success message
if result.acknowledged:
    print("Document inserted successfully.")




#   b. Implement a Python function that connects to a Cassandra database and inserts a new record into a table named "products". The record should contain fields like "id", "name", and "price". Handle any potential errors that may occur during the insertion


from cassandra.cluster import Cluster

def insert_product(id, name, price):
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()

    try:
        session.execute("INSERT INTO mykeyspace.products (id, name, price) VALUES (%s, %s, %s)",
                        (id, name, price))
        print("Record inserted successfully.")
    except Exception as e:
        print(f"Error occurred during insertion: {str(e)}")

    cluster.shutdown()


In [ ]:
# 2. Document-oriented NoSQL Databases:
#  a. Given a MongoDB collection named "books", write a Python function that fetches all the books published in the last year and prints their titles and authors.



from pymongo import MongoClient
from datetime import datetime, timedelta

def fetch_recent_books():
    client = MongoClient('mongodb://localhost:27017/')
    database = client['mydatabase']
    collection = database['books']

    last_year = datetime.now() - timedelta(days=365)
    recent_books = collection.find({'publish_date': {'$gte': last_year}})

    for book in recent_books:
        print(f"Title: {book['title']}, Author: {book['author']}")

    client.close()
    
    
    

#   b. Design a schema for a document-oriented NoSQL database to store customer information for an e-commerce platform. Write a Python program to insert a new customer document into the database and handle any necessary validations.



from pymongo import MongoClient

def insert_customer(customer_data):
    client = MongoClient('mongodb://localhost:27017/')
    database = client['mydatabase']
    collection = database['customers']

    # Perform necessary validations on customer_data

    # Insert the customer document
    result = collection.insert_one(customer_data)

    if result.acknowledged:
        print("Customer document inserted successfully.")

    client.close()




In [1]:
#  3. High Availability and Fault Tolerance:
#   a. Explain the concept of replica sets in MongoDB. Write a Python program that connects to a MongoDB replica set and retrieves the status of the primary and secondary nodes.




from pymongo import MongoClient

def retrieve_replica_set_status():
    client = MongoClient('mongodb://localhost:27017/')
    admin_db = client.admin

    # Run the isMaster command to retrieve replica set status
    result = admin_db.command('isMaster')

    # Primary node information
    print(f"Primary: {result['primary']}")

    # Secondary nodes information
    print("Secondaries:")
    for secondary in result['secondaries']:
        print(secondary)

    client.close()



#   b. Describe how Cassandra ensures high availability and fault tolerance in a distributed database system. Write a Python program that connects to a Cassandra cluster and fetches the status of the nodes.




from cassandra.cluster import Cluster

def fetch_node_status():
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()

    try:
        rows = session.execute("SELECT * FROM system.local")
        for row in rows:
            print(f"Node: {row['broadcast_address']}, Status: {row['status']}")
    except Exception as e:
        print(f"Error occurred during node status retrieval: {str(e)}")

    cluster.shutdown()




In [ ]:
# 4. Sharding in MongoDB:
#   a. Explain the concept of sharding in MongoDB and how it improves performance and scalability. Write a Python program that sets up sharding for a MongoDB cluster and inserts multiple documents into a sharded collection.




from pymongo import MongoClient

def setup_sharding():
    client = MongoClient('mongodb://localhost:27017/')
    admin_db = client.admin

    # Enable sharding on the database
    admin_db.command('enableSharding', 'mydatabase')

    # Shard the collection
    admin_db.command('shardCollection', 'mydatabase.students', key={'_id': 'hashed'})

    client.close()

def insert_documents():
    client = MongoClient('mongodb://localhost:27017/')
    database = client['mydatabase']
    collection = database['students']

    documents = [
        {'_id': 1, 'name': 'John Doe', 'age': 18, 'grade': 'A'},
        {'_id': 2, 'name': 'Jane Smith', 'age': 17, 'grade': 'B'},
        # Add more documents
    ]

    result = collection.insert_many(documents)

    if result.acknowledged:
        print("Documents inserted successfully.")

    client.close()

    
    
    
#   b. Design a sharding strategy for a social media application where user data needs to be distributed across multiple shards. Write a Python program to demonstrate how data is distributed and retrieved from the sharded cluster.



from pymongo import MongoClient

def insert_user(user_data):
    client = MongoClient('mongodb://localhost:27017/')
    database = client['mydatabase']
    collection = database['users']

    # Perform necessary validations on user_data

    result = collection.insert_one(user_data)

    if result.acknowledged:
        print("User document inserted successfully.")

    client.close()

def retrieve_users_by_username(username):
    client = MongoClient('mongodb://localhost:27017/')
    database = client['mydatabase']
    collection = database['users']

    query = {'username': {'$regex': f'^{username}.*'}}
    users = collection.find(query)

    for user in users:
        print(f"User ID: {user['_id']}, Username: {user['username']}")

    client.close()


In [ ]:
# 5. Indexing in MongoDB:
#   a. Describe the concept of indexing in MongoDB and its importance in query optimization. Write a Python program that creates an index on a specific field in a MongoDB collection and executes a query using that index.



from pymongo import MongoClient

def create_index_and_query():
    client = MongoClient('mongodb://localhost:27017/')
    database = client['mydatabase']
    collection = database['products']

    # Create an index on the 'name' field
    collection.create_index('name')

    # Execute a query using the index
    query = {'name': 'Example Product'}
    result = collection.find(query).explain()

    # Print the query execution details
    print(result['executionStats'])

    client.close()



#   b. Given a MongoDB collection named "products", write a Python function that searches for products with a specific keyword in the name or description. Optimize the query by adding appropriate indexes.



from pymongo import MongoClient

def search_products(keyword):
    client = MongoClient('mongodb://localhost:27017/')
    database = client['mydatabase']
    collection = database['products']

    # Create indexes on 'name' and 'description' fields
    collection.create_index('name')
    collection.create_index('description')

    # Execute the query using indexes
    query = {'$or': [{'name': {'$regex': keyword, '$options': 'i'}},
                     {'description': {'$regex': keyword, '$options': 'i'}}]}
    products = collection.find(query)

    for product in products:
        print(f"Product Name: {product['name']}, Description: {product['description']}")

    client.close()

    
    